In [28]:
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv

import os

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
model_name = os.environ.get("OPENAI_MODEL_NAME")

llm = OpenAI(api_key=api_key, model=model_name)

In [29]:
system_prompt_for_model_generation = """
You are an expert Python developer and OpenAPI/Swagger parser. You are given a Swagger/OpenAPI JSON file describing an API.
Your task is to extract and convert all schema definitions (under `components.schemas`) into idiomatic Python classes using
Pydantic (v1 or v2 as specified). Each model should reflect the correct type annotations and field-level validation based on the JSON Schema provided.

Follow these guidelines:
1. Use appropriate Pydantic types (`str`, `int`, `float`, `datetime`, `List`, `Dict`, `Optional`, etc.).
2. Map enums to Python `Enum` classes when `enum` is used.
3. Use `Field(...)` with `description=`, `default=`, `example=`, and other OpenAPI metadata where applicable.
4. Handle `$ref` fields correctly by referencing the appropriate class name.
5. Respect `required` fields vs `optional` fields.
6. Handle nested object types as nested Pydantic models.
7. If a model inherits from another (using `allOf` or `oneOf`), use proper Pydantic inheritance.
8. Flatten unused or redundant definitions (e.g., response-only wrappers).

Output format:
Generate valid Python code that:
- Contains only the model classes.
- Is formatted with imports at the top.
- Includes one model per class with comments summarizing the purpose based on schema `description`.

Advanced Ideas:
- Use custom field validators (e.g., for minLength, pattern, format).
- Add type hints for clarity (e.g., `Optional[List[str]] = None`).
- Sanitize or rename invalid Python identifiers from OpenAPI (`my-field` → `my_field`).
- Optionally group related models in namespaces or files (if relevant).
- Add `Config` class if `extra = "forbid"` or case rules are present.

Example:
Given:
```json
{
  "Pet": {
    "type": "object",
    "required": ["id", "name"],
    "properties": {
      "id": { "type": "integer", "format": "int64" },
      "name": { "type": "string" },
      "tag": { "type": "string" }
    }
  }
}```

Result:
from pydantic import BaseModel, Field
from typing import Optional

class Pet(BaseModel):
    ```Represents a pet with id, name, and optional tag.```
    id: int = Field(..., description="Unique identifier for the pet")
    name: str = Field(..., description="Name of the pet")
    tag: Optional[str] = Field(None, description="Tag of the pet")
"""

In [30]:
system_prompt_for_code_generation = """
You are an expert Python developer and OpenAPI parser. Your task is to generate functions that send HTTP requests to API endpoints described in
the provided Swagger/OpenAPI JSON. You must use the Pydantic models (previously generated or generated within the prompt) for both requests and responses.

🔹 Follow these rules:

1. For each endpoint (path + method), generate a Python function:
    - The function name should be snake_case, derived from the operationId or path.
    - Use `httpx` or `requests` (choose one and stay consistent).
    - Include all path, query, and header parameters.
    - Use Pydantic models for request bodies (for POST, PUT, PATCH).
    - Use Pydantic models for parsing responses if schemas are defined.
  
2. Base URL:
    - Hardcode the base URL as a module-level constant (e.g., `BASE_URL = "https://api.example.com"`).
    - Do not pass it as a parameter.
    - Use the base BASE_URL in every function to construct the full URL.

3. Handle authentication:
    - If the endpoint requires security (e.g., `BearerAuth`, `ApiKeyAuth`), add an `auth_token` parameter to the function.
    - Add appropriate headers automatically (e.g., `Authorization: Bearer <token>`).

4. Include rich docstrings:
    - Describe what the function does.
    - Explain each parameter (name, type, required/optional, and description from the schema).
    - Specify the return type and how errors are handled.

5. Use type hints for all parameters and return values.
6. Functions must import all required modules and be self-contained.
7. Optionally group functions by tag or endpoint category.
8. Use `.raise_for_status()` to handle errors unless otherwise specified.

Advanced ideas:
- Auto-handle enum validation, nested schemas, and response unions.
- Add built-in retry or timeout config in the request client.
- Wrap the whole API in a class like `MyApiClient` with shared logic and token injection.
- If `operationId` is missing, generate function name from method + path.

Example:

Given:
```json
"/pets/{petId}": {
  "get": {
    "operationId": "getPetById",
    "parameters": [
      {
        "name": "petId",
        "in": "path",
        "required": true,
        "schema": { "type": "integer", "format": "int64" }
      }
    ],
    "responses": {
      "200": {
        "description": "Successful response",
        "content": {
          "application/json": {
            "schema": { "$ref": "#/components/schemas/Pet" }
          }
        }
      }
    },
    "security": [{ "BearerAuth": [] }]
  }
}
```

Result (assume that the Pet model is already generated in this file):
```
import requests

BASE_URL = "https://api.example.com"

def get_pet_by_id(pet_id: int, auth_token: str) -> Pet:
    ```
    Retrieve a pet by its ID.

    Args:
        pet_id (int): The ID of the pet to retrieve.
        auth_token (str): Bearer token for authentication.

    Returns:
        Pet: A Pydantic model representing the retrieved pet.

    Raises:
        requests.HTTPError: If the response status is not successful.
    ```
    url = f"{BASE_URL}/pets/{pet_id}"
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return Pet(**response.json())
```
"""

In [37]:
import json

swagger_path = "swagger.json"
with open(swagger_path, "r") as f:
    swagger_json = json.load(f)


In [36]:
def extract_schemes() -> dict:
    return swagger_json.get("components", {}).get("schemas", {})

schemes = extract_schemes()


In [33]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content=system_prompt_for_model_generation),
    ChatMessage(role="user", content=f"here are the schemes: {schemes}")
]

model_generation_result = llm.chat(messages)



In [34]:
messages = [
    ChatMessage(role="system", content=system_prompt_for_code_generation),
    ChatMessage(role="user", content=f"here is the swagger.json: {swagger_json}")
]

functions_generation_result = llm.chat(messages)



In [35]:
full_code = f"""
{model_generation_result}

{functions_generation_result}
"""